In [69]:
from omegaconf import OmegaConf
import os
import torch

import hydra
from omegaconf import DictConfig, OmegaConf
from models.together_model import ProteinVAELLMmodel
from data import all_atom
# Pytorch lightning imports
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

from data.pdb_dataloader import PdbDataModule
from models.flow_module import FlowModule
from experiments import utils as eu
import openfold.utils.rigid_utils as ru
import torch.nn.functional as F
cfg = OmegaConf.load("configs/base.yaml")
_cfg = cfg
_data_cfg = cfg.data
_exp_cfg = cfg.experiment
_datamodule: LightningDataModule = PdbDataModule(_data_cfg)
_datamodule.setup(stage="fit")

# Set up stuff

In [70]:
cfg = OmegaConf.load("configs/base.yaml")
_cfg = cfg
_data_cfg = cfg.data
_exp_cfg = cfg.experiment
training_cfg = _exp_cfg.training
_datamodule: LightningDataModule = PdbDataModule(_data_cfg)
_datamodule.setup(stage="fit")

In [71]:
train_loader = _datamodule.train_dataloader()

/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [72]:
from data.my_interpolant import Interpolant 
interpolant = Interpolant(cfg.interpolant)
interpolant.set_device("cuda")

In [73]:
# model = ProteinVAELLMmodel(cfg).to("cuda")
# model.attach_backward_hooks() # attach backward hooks

from models.vaellm_model import VAE_GPT2
from transformers import GPT2Model

gpt2 = GPT2Model.from_pretrained('gpt2')
# gpt2 = GPT2Model(GPT2Config())
gpt2.wte=None
vaellm_model = VAE_GPT2(base_model=gpt2, emb_dim=768, z_dim=768).to("cuda")

In [74]:
optimizer = torch.optim.AdamW(
            params=vaellm_model.parameters(),
            lr=0.0001)

# Helper functions

In [75]:
def _get_bb_reps(noisy_batch):
    '''
        This function prepares the trajectories into representations for the LLM.
    '''
    # Convert rotmats to quaternions (4D)
    quats = ru.rot_to_quat(noisy_batch['rotmats_t'])

    # Reduce quaternions to 3D
    condition = quats[..., 0] < 0
    quats[condition] *= -1
    quats = quats[..., 1:] # [B, l, N, 3]
        
    # Concatenate with translation vectors
    concat_quats_trans = torch.cat((quats, noisy_batch['trans_t']), dim=-1) # [B, l, N, 6]

    # Concatenate residues to form backbones
    backbone = concat_quats_trans.reshape(*concat_quats_trans.shape[:-2], -1) # [B, l, 6N]

    return backbone

def _recover_rots_trans( z):
    '''
        This function recovers the rotation matrices and translation vectors from the vector representation.

        Parameters
        ----------
        z:          Tensor, [B, l, max_num_res, 6]

        Returns
        -------
        recovered_rots:
            Tensor, [B, l, max_num_res, 3, 3]
        recovered_trans:
            Tensor, [B, l, max_num_res, 3]
    '''
    B, l = z.shape[0], z.shape[1]
    # print("z shape: ", z.shape)
    recovered_reduced_quats = z[:, :, :, :3]
    recovered_trans = z[:, :, :, 3:]
    ones = torch.ones(B, l, z.shape[2], 1).to(recovered_trans.device) # TODO will need to change here
    recovered_quats = torch.cat((ones, recovered_reduced_quats), dim=-1)
    recovered_quats = F.normalize(recovered_quats, p=2, dim=-1) # normalize (1,x,y,z) as AlphaFold did

    recovered_rots = ru.quat_to_rot(recovered_quats)

    return recovered_rots, recovered_trans

# Train

In [66]:
batch = next(iter(train_loader))
print(batch["aatype"])
for key, value in batch.items():
    batch[key] = value.to("cuda")
noisy_batch = interpolant.corrupt_batch(batch)

tensor([[19, 16, 10, 10,  6,  5,  2, 14,  1, 17,  1, 10, 19, 14,  1,  7,  5,  0,
         19,  2, 10,  1,  4,  9, 10, 11,  2, 15,  5, 18, 14, 17, 12, 15, 17, 18,
          5,  5,  3, 10,  5, 11,  5, 10,  5, 17, 10, 13, 16, 10,  1, 15, 14,  7,
          3, 11,  6, 19, 11, 15, 10, 14,  7,  0,  3, 18, 10,  0, 16,  1, 19, 16,
          3, 16,  6, 10,  1, 10,  5, 19,  0,  2, 12, 15,  5,  7,  1, 16, 10, 18,
          4, 16,  4, 15,  0,  0, 14,  3, 17,  7,  0, 15,  0,  6, 16, 10, 18, 13,
          7, 15,  7, 16,  1, 10, 16, 19, 10],
        [12, 11, 18, 19,  3,  7, 13, 19, 19,  0, 19, 14,  0,  3, 11, 11,  3,  0,
         18,  1,  6, 12,  0,  0, 11,  0,  0, 14, 10, 13, 11,  6, 13,  7,  0, 10,
          1,  9, 19,  6,  4, 17,  0, 15,  3, 19, 14,  3,  7, 11, 19, 16,  3, 13,
          1, 12,  0, 19, 11,  0,  6,  6,  2,  6,  6, 19, 19, 13, 15, 17,  9,  6,
         18, 14, 15, 11,  6, 19,  1,  3,  0,  0,  2,  5, 11, 12, 12, 15,  3, 14,
          1, 12, 11,  6, 13,  7,  6, 15, 12, 14, 13,  3,  7, 11

In [67]:
B, l, num_res = noisy_batch['res_mask'].shape
loss_mask = noisy_batch['res_mask'].reshape(B*l, num_res)
        
if training_cfg.min_plddt_mask is not None:
    plddt_mask = noisy_batch['res_plddt'] > training_cfg.min_plddt_mask
    loss_mask *= plddt_mask
        

# Ground truth labels
gt_trans = noisy_batch['trans_t']
gt_rotmats = noisy_batch['rotmats_t'] 

# Model output predictions.
backbone = _get_bb_reps(noisy_batch)
llm_out = vaellm_model(backbone)
z = llm_out["z_sampled"].reshape(B, l, 128, -1)
recovered_rots, recovered_trans = _recover_rots_trans(z) # [B, l, N, 3, 3], [B, l, N, 3]
pred_trans = recovered_trans
pred_rotmats = recovered_rots
        
# Shift for CausalLM
shifted_pred_trans  = pred_trans[:, :-1, :, :]
shifted_pred_rotmats = pred_rotmats[:, :-1, :, :, :]
shifted_gt_trans = gt_trans[:, 1:, :, :]
shifted_gt_rotmats = gt_rotmats[:, 1:, :, :, :]

# Reshape back to [B*(l-1), 128, *]
flat_shifted_pred_trans = shifted_pred_trans.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3)
flat_shifted_pred_rotmats = shifted_pred_rotmats.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3, 3)

flat_shifted_gt_trans = shifted_gt_trans.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3)
flat_shifted_gt_rotmats = shifted_gt_rotmats.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3, 3)

# Timestep used for normalization.
t = noisy_batch['t'].reshape(B, l, 1)[:, 1:, :].reshape(-1,1) # We throw away the first time points
norm_scale = 1 - torch.min(
    t[..., None], torch.tensor(training_cfg.t_normalize_clip))
        
        

# Backbone atom loss
gt_bb_atoms = all_atom.to_atom37(flat_shifted_gt_trans, flat_shifted_gt_rotmats)[:, :, :3] 
pred_bb_atoms = all_atom.to_atom37(flat_shifted_pred_trans, flat_shifted_pred_rotmats)[:, :, :3]

gt_bb_atoms *= training_cfg.bb_atom_scale / norm_scale[..., None]
pred_bb_atoms *= training_cfg.bb_atom_scale / norm_scale[..., None]
                
        
loss_denom = torch.sum(loss_mask, dim=-1, dtype=torch.float).mean() * 3 # Added a mean here, this doesn'y matter since our mask is all 1's
bb_atom_loss = torch.sum(
    (gt_bb_atoms - pred_bb_atoms) ** 2,
    dim=(-1, -2, -3)
) / loss_denom

# Pairwise distance loss
num_batch = gt_bb_atoms.shape[0]
gt_flat_atoms = gt_bb_atoms.reshape([num_batch, num_res*3, 3])
gt_pair_dists = torch.linalg.norm(
    gt_flat_atoms[:, :, None, :] - gt_flat_atoms[:, None, :, :], dim=-1)
pred_flat_atoms = pred_bb_atoms.reshape([num_batch, num_res*3, 3])
pred_pair_dists = torch.linalg.norm(
    pred_flat_atoms[:, :, None, :] - pred_flat_atoms[:, None, :, :], dim=-1)

flat_loss_mask = torch.tile(loss_mask[:, :, None], (1, 1, 3))[B:,:,:] # change the shape because we shifted tokens, all entries of loss masks are 1 so don't matter, we throw away B tokens
flat_loss_mask = flat_loss_mask.reshape([num_batch, num_res*3])
flat_res_mask = torch.tile(loss_mask[:, :, None], (1, 1, 3))[B:,:,:] # change the shape because we shifted tokens, all entries of loss masks are 1 so don't matter
flat_res_mask = flat_res_mask.reshape([num_batch, num_res*3])

gt_pair_dists = gt_pair_dists * flat_loss_mask[..., None]
pred_pair_dists = pred_pair_dists * flat_loss_mask[..., None]
pair_dist_mask = flat_loss_mask[..., None] * flat_res_mask[:, None, :]

dist_mat_loss = torch.sum(
    (gt_pair_dists - pred_pair_dists)**2 * pair_dist_mask,
    dim=(1, 2))
dist_mat_loss /= (torch.sum(pair_dist_mask, dim=(1, 2)) - num_res)

auxiliary_loss = (bb_atom_loss + dist_mat_loss) * (
    t[:, 0]> training_cfg.aux_loss_t_pass
)
auxiliary_loss *= _exp_cfg.training.aux_loss_weight
        
auxiliary_loss = auxiliary_loss.mean()
kl_div = (1 + 2 * llm_out["log_sigma"] - llm_out["mu"].pow(2) - llm_out["log_sigma"].exp().pow(2))[:, :-1, :] # Throw away the last mu and sigma because we're not using it to predict
kl_div = - 0.5 * kl_div.sum(dim=-1).mean()
mse_loss = F.mse_loss(flat_shifted_pred_trans, flat_shifted_gt_trans) + F.mse_loss(flat_shifted_pred_rotmats, flat_shifted_gt_rotmats)

In [68]:
loss = mse_loss + auxiliary_loss + kl_div
print(loss)
optimizer.zero_grad()
loss.backward()
optimizer.step()

tensor(2345.4954, device='cuda:0', grad_fn=<AddBackward0>)


# Put stuff together

In [77]:
from tqdm import tqdm

for batch in train_loader:
    for key, value in batch.items():
        batch[key] = value.to("cuda")
    noisy_batch = interpolant.corrupt_batch(batch)

    B, l, num_res = noisy_batch['res_mask'].shape
    loss_mask = noisy_batch['res_mask'].reshape(B*l, num_res)
            
    if training_cfg.min_plddt_mask is not None:
        plddt_mask = noisy_batch['res_plddt'] > training_cfg.min_plddt_mask
        loss_mask *= plddt_mask
            

    # Ground truth labels
    gt_trans = noisy_batch['trans_t']
    gt_rotmats = noisy_batch['rotmats_t'] 

    # Model output predictions.
    backbone = _get_bb_reps(noisy_batch)
    llm_out = vaellm_model(backbone)
    z = llm_out["z_sampled"].reshape(B, l, 128, -1)
    recovered_rots, recovered_trans = _recover_rots_trans(z) # [B, l, N, 3, 3], [B, l, N, 3]
    pred_trans = recovered_trans
    pred_rotmats = recovered_rots
            
    # Shift for CausalLM
    shifted_pred_trans  = pred_trans[:, :-1, :, :]
    shifted_pred_rotmats = pred_rotmats[:, :-1, :, :, :]
    shifted_gt_trans = gt_trans[:, 1:, :, :]
    shifted_gt_rotmats = gt_rotmats[:, 1:, :, :, :]

    # Reshape back to [B*(l-1), 128, *]
    flat_shifted_pred_trans = shifted_pred_trans.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3)
    flat_shifted_pred_rotmats = shifted_pred_rotmats.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3, 3)

    flat_shifted_gt_trans = shifted_gt_trans.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3)
    flat_shifted_gt_rotmats = shifted_gt_rotmats.reshape(B*(l-1), _data_cfg.dataset.max_num_res, 3, 3)

    # Timestep used for normalization.
    t = noisy_batch['t'].reshape(B, l, 1)[:, 1:, :].reshape(-1,1) # We throw away the first time points
    norm_scale = 1 - torch.min(
        t[..., None], torch.tensor(training_cfg.t_normalize_clip))
            
            

    # Backbone atom loss
    gt_bb_atoms = all_atom.to_atom37(flat_shifted_gt_trans, flat_shifted_gt_rotmats)[:, :, :3] 
    pred_bb_atoms = all_atom.to_atom37(flat_shifted_pred_trans, flat_shifted_pred_rotmats)[:, :, :3]

    gt_bb_atoms *= training_cfg.bb_atom_scale / norm_scale[..., None]
    pred_bb_atoms *= training_cfg.bb_atom_scale / norm_scale[..., None]
                    
            
    loss_denom = torch.sum(loss_mask, dim=-1, dtype=torch.float).mean() * 3 # Added a mean here, this doesn'y matter since our mask is all 1's
    bb_atom_loss = torch.sum(
        (gt_bb_atoms - pred_bb_atoms) ** 2,
        dim=(-1, -2, -3)
    ) / loss_denom

    # Pairwise distance loss
    num_batch = gt_bb_atoms.shape[0]
    gt_flat_atoms = gt_bb_atoms.reshape([num_batch, num_res*3, 3])
    gt_pair_dists = torch.linalg.norm(
        gt_flat_atoms[:, :, None, :] - gt_flat_atoms[:, None, :, :], dim=-1)
    pred_flat_atoms = pred_bb_atoms.reshape([num_batch, num_res*3, 3])
    pred_pair_dists = torch.linalg.norm(
        pred_flat_atoms[:, :, None, :] - pred_flat_atoms[:, None, :, :], dim=-1)

    flat_loss_mask = torch.tile(loss_mask[:, :, None], (1, 1, 3))[B:,:,:] # change the shape because we shifted tokens, all entries of loss masks are 1 so don't matter, we throw away B tokens
    flat_loss_mask = flat_loss_mask.reshape([num_batch, num_res*3])
    flat_res_mask = torch.tile(loss_mask[:, :, None], (1, 1, 3))[B:,:,:] # change the shape because we shifted tokens, all entries of loss masks are 1 so don't matter
    flat_res_mask = flat_res_mask.reshape([num_batch, num_res*3])

    gt_pair_dists = gt_pair_dists * flat_loss_mask[..., None]
    pred_pair_dists = pred_pair_dists * flat_loss_mask[..., None]
    pair_dist_mask = flat_loss_mask[..., None] * flat_res_mask[:, None, :]

    dist_mat_loss = torch.sum(
        (gt_pair_dists - pred_pair_dists)**2 * pair_dist_mask,
        dim=(1, 2))
    dist_mat_loss /= (torch.sum(pair_dist_mask, dim=(1, 2)) - num_res)

    auxiliary_loss = (bb_atom_loss + dist_mat_loss) * (
        t[:, 0]> training_cfg.aux_loss_t_pass
    )
    auxiliary_loss *= _exp_cfg.training.aux_loss_weight
            
    auxiliary_loss = auxiliary_loss.mean()
    kl_div = (1 + 2 * llm_out["log_sigma"] - llm_out["mu"].pow(2) - llm_out["log_sigma"].exp().pow(2))[:, :-1, :] # Throw away the last mu and sigma because we're not using it to predict
    kl_div = - 0.5 * kl_div.sum(dim=-1).mean()
    mse_loss = F.mse_loss(flat_shifted_pred_trans, flat_shifted_gt_trans) + F.mse_loss(flat_shifted_pred_rotmats, flat_shifted_gt_rotmats)

    loss = mse_loss + auxiliary_loss + kl_div
    print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0it [00:00, ?it/s]/gpfs/gibbs/project/dijk/sh2748/conda_envs/fm/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
3it [00:02,  1.80it/s]

tensor(12725998., device='cuda:0', grad_fn=<AddBackward0>)
tensor(663018.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(448949.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(172587.1250, device='cuda:0', grad_fn=<AddBackward0>)


7it [00:02,  4.88it/s]

tensor(107329.7109, device='cuda:0', grad_fn=<AddBackward0>)
tensor(36836.4727, device='cuda:0', grad_fn=<AddBackward0>)
tensor(38472.6172, device='cuda:0', grad_fn=<AddBackward0>)


11it [00:02,  8.24it/s]

...

tensor(256.8788, device='cuda:0', grad_fn=<AddBackward0>)
tensor(154.9612, device='cuda:0', grad_fn=<AddBackward0>)
tensor(146.4446, device='cuda:0', grad_fn=<AddBackward0>)


2428it [02:52, 12.01it/s]

tensor(264.0884, device='cuda:0', grad_fn=<AddBackward0>)
tensor(197.7039, device='cuda:0', grad_fn=<AddBackward0>)


2430it [02:52,  9.29it/s]

tensor(239.7518, device='cuda:0', grad_fn=<AddBackward0>)
tensor(175.4806, device='cuda:0', grad_fn=<AddBackward0>)
